In [8]:
import pandas as pd

df = pd.read_csv("D:/Kaggle/SalesPrice/Titanic/train.csv", index_col=False)

In [12]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [14]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [18]:
df = df.drop(["Cabin"], axis=1)

In [19]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         2
dtype: int64

In [20]:
import numpy as np

num_df = df.select_dtypes(include = np.number)
cat_df = df.select_dtypes(include = np.object_)

In [21]:
num_df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Age            177
SibSp            0
Parch            0
Fare             0
dtype: int64

In [25]:
cat_df.isna().sum()

Name        0
Sex         0
Ticket      0
Embarked    2
dtype: int64

In [28]:
cat_df["Embarked"] = cat_df["Embarked"].fillna("S")

In [29]:
cat_df.groupby(["Embarked"]).count()

,Name,Sex,Ticket
Embarked,,,
C,168,168,168
Q,77,77,77
S,646,646,646


In [30]:
cat_df.isna().sum()

Name        0
Sex         0
Ticket      0
Embarked    0
dtype: int64

In [32]:
cat_df = cat_df.drop(["Name","Ticket"], axis=1)

In [33]:
cat_df.isna().sum()

Sex         0
Embarked    0
dtype: int64

In [35]:
from sklearn.preprocessing import LabelEncoder


categorical_columns = cat_df.columns.tolist()

# Label Encoding
label_encoders = {}
for column in categorical_columns:
    le = LabelEncoder()
    cat_df[column] = le.fit_transform(cat_df[column])
    label_encoders[column] = le

print(cat_df.head())

   Sex  Embarked
0    1         2
1    0         0
2    0         2
3    0         2
4    1         2


In [37]:
cat_df = cat_df.astype(int)

In [39]:
mean = num_df["Age"].mean(skipna=True) 

mean

29.69911764705882

In [57]:
num_df = df.select_dtypes(include = np.number)

In [58]:
num_df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Age            177
SibSp            0
Parch            0
Fare             0
dtype: int64

In [59]:
mean

29.69911764705882

In [60]:
num_df.fillna(mean, inplace=True)

In [61]:
num_df.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Age            0
SibSp          0
Parch          0
Fare           0
dtype: int64

In [66]:
target_df = pd.DataFrame()
target_df["Survived"] = pd.DataFrame(num_df["Survived"])

In [68]:
target_df

,Survived
0,0
1,1
2,1
3,1
4,0
...,...
886,0
887,1
888,0
889,1


In [69]:
num_df = num_df.drop(["Survived"],axis=1)

In [72]:
df_train = pd.concat([num_df.reset_index(drop=True),cat_df.reset_index(drop=True),target_df.reset_index(drop=True)], axis=1)

In [73]:
df_train

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked,Survived
0,1,3,22.000000,1,0,7.2500,1,2,0
1,2,1,38.000000,1,0,71.2833,0,0,1
2,3,3,26.000000,0,0,7.9250,0,2,1
3,4,1,35.000000,1,0,53.1000,0,2,1
4,5,3,35.000000,0,0,8.0500,1,2,0
...,...,...,...,...,...,...,...,...,...
886,887,2,27.000000,0,0,13.0000,1,2,0
887,888,1,19.000000,0,0,30.0000,0,2,1
888,889,3,29.699118,1,2,23.4500,0,2,0
889,890,1,26.000000,0,0,30.0000,1,0,1


In [74]:
import statsmodels.api as sm

df_train = df_train.astype(int)

# Define the target variable and features
X = df_train.drop('Survived', axis=1)
y = df_train['Survived']

# Add a constant term for the intercept
X = sm.add_constant(X)

# Fit the linear regression model
model = sm.OLS(y, X).fit()

# Get the summary of the regression model
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:               Survived   R-squared:                       0.398
Model:                            OLS   Adj. R-squared:                  0.392
Method:                 Least Squares   F-statistic:                     72.84
Date:                Wed, 17 Jul 2024   Prob (F-statistic):           6.54e-92
Time:                        12:36:27   Log-Likelihood:                -395.99
No. Observations:                 891   AIC:                             810.0
Df Residuals:                     882   BIC:                             853.1
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           1.3414      0.077     17.494      

In [75]:
pid = pd.DataFrame()
pid["PassengerId"] = pd.DataFrame(df_train["PassengerId"])

In [78]:
df_train = df_train.drop(["PassengerId","Parch","Fare"],axis=1)


In [87]:
import statsmodels.api as sm

# Define the target variable and features
X = df_train.drop(['Survived'], axis=1)
y = df_train['Survived']

# Add a constant term for the intercept
X = sm.add_constant(X)

# Fit the linear regression model
model = sm.OLS(y, X).fit()

# Get the summary of the regression model
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:               Survived   R-squared:                       0.397
Model:                            OLS   Adj. R-squared:                  0.393
Method:                 Least Squares   F-statistic:                     116.5
Date:                Wed, 17 Jul 2024   Prob (F-statistic):           1.21e-94
Time:                        12:49:35   Log-Likelihood:                -396.71
No. Observations:                 891   AIC:                             805.4
Df Residuals:                     885   BIC:                             834.2
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.3728      0.061     22.430      0.0

In [82]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

df_encoded = pd.get_dummies(df_train, drop_first=True)

# Define the target variable and features
X = df_encoded.drop('Survived', axis=1)  # replace 'Target' with your actual target column name
y = df_encoded['Survived']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and fit the Random Forest classifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

In [85]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

print('Classification Report:')
print(classification_report(y_test, y_pred))

print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.8156424581005587
Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.88      0.85       105
           1       0.81      0.73      0.77        74

    accuracy                           0.82       179
   macro avg       0.81      0.80      0.81       179
weighted avg       0.82      0.82      0.81       179

Confusion Matrix:
[[92 13]
 [20 54]]


In [92]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Initialize the Min-Max Scaler
scaler = MinMaxScaler()

# Normalize the DataFrame
titanic_normalized = pd.DataFrame(scaler.fit_transform(df_train), columns=df_train.columns)

# Display the first few rows of the normalized data
print(titanic_normalized.head())

   Pclass     Age  SibSp  Sex  Embarked  Survived
0     1.0  0.2750  0.125  1.0       1.0       0.0
1     0.0  0.4750  0.125  0.0       0.0       1.0
2     1.0  0.3250  0.000  0.0       1.0       1.0
3     0.0  0.4375  0.125  0.0       1.0       1.0
4     1.0  0.4375  0.000  1.0       1.0       0.0


In [93]:
import statsmodels.api as sm

# Define the target variable and features
X = titanic_normalized.drop(['Survived'], axis=1)
y = titanic_normalized['Survived']

# Add a constant term for the intercept
X = sm.add_constant(X)

# Fit the linear regression model
model = sm.OLS(y, X).fit()

# Get the summary of the regression model
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:               Survived   R-squared:                       0.397
Model:                            OLS   Adj. R-squared:                  0.393
Method:                 Least Squares   F-statistic:                     116.5
Date:                Wed, 17 Jul 2024   Prob (F-statistic):           1.21e-94
Time:                        12:58:56   Log-Likelihood:                -396.71
No. Observations:                 891   AIC:                             805.4
Df Residuals:                     885   BIC:                             834.2
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1943      0.051     23.350      0.0

In [94]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Define the target variable and features
X = titanic_normalized.drop('Survived', axis=1)  # replace 'Target' with your actual target column name
y = titanic_normalized['Survived']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and fit the Random Forest classifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

In [95]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

print('Classification Report:')
print(classification_report(y_test, y_pred))

print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.8212290502793296
Classification Report:
              precision    recall  f1-score   support

         0.0       0.83      0.88      0.85       105
         1.0       0.81      0.74      0.77        74

    accuracy                           0.82       179
   macro avg       0.82      0.81      0.81       179
weighted avg       0.82      0.82      0.82       179

Confusion Matrix:
[[92 13]
 [19 55]]


In [96]:
df_train = pd.concat([num_df.reset_index(drop=True),cat_df.reset_index(drop=True),target_df.reset_index(drop=True)], axis=1)

In [97]:
df_train

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked,Survived
0,1,3,22.000000,1,0,7.2500,1,2,0
1,2,1,38.000000,1,0,71.2833,0,0,1
2,3,3,26.000000,0,0,7.9250,0,2,1
3,4,1,35.000000,1,0,53.1000,0,2,1
4,5,3,35.000000,0,0,8.0500,1,2,0
...,...,...,...,...,...,...,...,...,...
886,887,2,27.000000,0,0,13.0000,1,2,0
887,888,1,19.000000,0,0,30.0000,0,2,1
888,889,3,29.699118,1,2,23.4500,0,2,0
889,890,1,26.000000,0,0,30.0000,1,0,1


In [98]:
df_train = df_train.drop(["PassengerId"],axis=1)

In [99]:
df_train

,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked,Survived
0,3,22.000000,1,0,7.2500,1,2,0
1,1,38.000000,1,0,71.2833,0,0,1
2,3,26.000000,0,0,7.9250,0,2,1
3,1,35.000000,1,0,53.1000,0,2,1
4,3,35.000000,0,0,8.0500,1,2,0
...,...,...,...,...,...,...,...,...
886,2,27.000000,0,0,13.0000,1,2,0
887,1,19.000000,0,0,30.0000,0,2,1
888,3,29.699118,1,2,23.4500,0,2,0
889,1,26.000000,0,0,30.0000,1,0,1


In [102]:
# Initialize the Standard Scaler
scaler = StandardScaler()
df = df_train.drop(["Survived"],axis=1)
# Standardize the DataFrame
titanic_standardized = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

# Display the first few rows of the standardized data
print(titanic_standardized.head())

     Pclass       Age     SibSp     Parch      Fare       Sex  Embarked
0  0.827377 -0.592481  0.432793 -0.473674 -0.502445  0.737695  0.585954
1 -1.566107  0.638789  0.432793 -0.473674  0.786845 -1.355574 -1.942303
2  0.827377 -0.284663 -0.474545 -0.473674 -0.488854 -1.355574  0.585954
3 -1.566107  0.407926  0.432793 -0.473674  0.420730 -1.355574  0.585954
4  0.827377  0.407926 -0.474545 -0.473674 -0.486337  0.737695  0.585954


In [103]:
df_train = pd.concat([titanic_standardized.reset_index(drop=True),target_df.reset_index(drop=True)], axis=1)

In [104]:
df_train

,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked,Survived
0,0.827377,-0.592481,0.432793,-0.473674,-0.502445,0.737695,0.585954,0
1,-1.566107,0.638789,0.432793,-0.473674,0.786845,-1.355574,-1.942303,1
2,0.827377,-0.284663,-0.474545,-0.473674,-0.488854,-1.355574,0.585954,1
3,-1.566107,0.407926,0.432793,-0.473674,0.420730,-1.355574,0.585954,1
4,0.827377,0.407926,-0.474545,-0.473674,-0.486337,0.737695,0.585954,0
...,...,...,...,...,...,...,...,...
886,-0.369365,-0.207709,-0.474545,-0.473674,-0.386671,0.737695,0.585954,0
887,-1.566107,-0.823344,-0.474545,-0.473674,-0.044381,-1.355574,0.585954,1
888,0.827377,0.000000,0.432793,2.008933,-0.176263,-1.355574,0.585954,0
889,-1.566107,-0.284663,-0.474545,-0.473674,-0.044381,0.737695,-1.942303,1


In [105]:
import statsmodels.api as sm

# Define the target variable and features
X = df_train.drop(['Survived'], axis=1)
y = df_train['Survived']

# Add a constant term for the intercept
X = sm.add_constant(X)

# Fit the linear regression model
model = sm.OLS(y, X).fit()

# Get the summary of the regression model
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:               Survived   R-squared:                       0.398
Model:                            OLS   Adj. R-squared:                  0.393
Method:                 Least Squares   F-statistic:                     83.36
Date:                Wed, 17 Jul 2024   Prob (F-statistic):           6.60e-93
Time:                        13:09:17   Log-Likelihood:                -395.94
No. Observations:                 891   AIC:                             807.9
Df Residuals:                     883   BIC:                             846.2
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3838      0.013     30.226      0.0

In [113]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Define the target variable and features
X = df_train.drop(['Survived','Parch','Fare'], axis=1)  # replace 'Target' with your actual target column name
y = df_train['Survived']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and fit the Random Forest classifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

print('Classification Report:')
print(classification_report(y_test, y_pred))

print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.8212290502793296
Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.89      0.85       105
           1       0.82      0.73      0.77        74

    accuracy                           0.82       179
   macro avg       0.82      0.81      0.81       179
weighted avg       0.82      0.82      0.82       179

Confusion Matrix:
[[93 12]
 [20 54]]


In [115]:
from sklearn.tree import DecisionTreeClassifier

# Define the target variable and features
X = df_train.drop(['Survived','Parch','Fare'], axis=1)
y = df_train['Survived']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Initialize and fit the Decision Tree classifier
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

print('Classification Report:')
print(classification_report(y_test, y_pred))

print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.776536312849162
Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.86      0.82       105
           1       0.77      0.66      0.71        74

    accuracy                           0.78       179
   macro avg       0.77      0.76      0.76       179
weighted avg       0.78      0.78      0.77       179

Confusion Matrix:
[[90 15]
 [25 49]]


In [116]:
df_train

,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked,Survived
0,0.827377,-0.592481,0.432793,-0.473674,-0.502445,0.737695,0.585954,0
1,-1.566107,0.638789,0.432793,-0.473674,0.786845,-1.355574,-1.942303,1
2,0.827377,-0.284663,-0.474545,-0.473674,-0.488854,-1.355574,0.585954,1
3,-1.566107,0.407926,0.432793,-0.473674,0.420730,-1.355574,0.585954,1
4,0.827377,0.407926,-0.474545,-0.473674,-0.486337,0.737695,0.585954,0
...,...,...,...,...,...,...,...,...
886,-0.369365,-0.207709,-0.474545,-0.473674,-0.386671,0.737695,0.585954,0
887,-1.566107,-0.823344,-0.474545,-0.473674,-0.044381,-1.355574,0.585954,1
888,0.827377,0.000000,0.432793,2.008933,-0.176263,-1.355574,0.585954,0
889,-1.566107,-0.284663,-0.474545,-0.473674,-0.044381,0.737695,-1.942303,1


In [118]:
from xgboost import XGBClassifier

X = df_train.drop(['Survived','Fare'], axis=1)
y = df_train['Survived']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and fit the XGBoost classifier
model = XGBClassifier(random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

print('Classification Report:')
print(classification_report(y_test, y_pred))

print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.8156424581005587
Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.87      0.85       105
           1       0.80      0.74      0.77        74

    accuracy                           0.82       179
   macro avg       0.81      0.80      0.81       179
weighted avg       0.81      0.82      0.81       179

Confusion Matrix:
[[91 14]
 [19 55]]


In [119]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Define the target variable and features
X = df_train.drop(['Survived','Parch','Fare'], axis=1)  # replace 'Target' with your actual target column name
y = df_train['Survived']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and fit the Random Forest classifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

print('Classification Report:')
print(classification_report(y_test, y_pred))

print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.8212290502793296
Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.89      0.85       105
           1       0.82      0.73      0.77        74

    accuracy                           0.82       179
   macro avg       0.82      0.81      0.81       179
weighted avg       0.82      0.82      0.82       179

Confusion Matrix:
[[93 12]
 [20 54]]


In [120]:
test_df = pd.read_csv("D:/Kaggle/SalesPrice/Titanic/test.csv")

In [122]:
test_df = test_df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

In [123]:
test_df

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,34.5,0,0,7.8292,Q
1,3,female,47.0,1,0,7.0000,S
2,2,male,62.0,0,0,9.6875,Q
3,3,male,27.0,0,0,8.6625,S
4,3,female,22.0,1,1,12.2875,S
...,...,...,...,...,...,...,...
413,3,male,NaN,0,0,8.0500,S
414,1,female,39.0,0,0,108.9000,C
415,3,male,38.5,0,0,7.2500,S
416,3,male,NaN,0,0,8.0500,S


In [124]:
test_df['Age'].fillna(test_df['Age'].median(), inplace=True)
test_df['Embarked'].fillna(test_df['Embarked'].mode()[0], inplace=True)

C:\Users\vaibh\AppData\Local\Temp\ipykernel_24988\2190171194.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_df['Age'].fillna(test_df['Age'].median(), inplace=True)
C:\Users\vaibh\AppData\Local\Temp\ipykernel_24988\2190171194.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a co

In [126]:
test_df

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,34.5,0,0,7.8292,Q
1,3,female,47.0,1,0,7.0000,S
2,2,male,62.0,0,0,9.6875,Q
3,3,male,27.0,0,0,8.6625,S
4,3,female,22.0,1,1,12.2875,S
...,...,...,...,...,...,...,...
413,3,male,27.0,0,0,8.0500,S
414,1,female,39.0,0,0,108.9000,C
415,3,male,38.5,0,0,7.2500,S
416,3,male,27.0,0,0,8.0500,S
